In [2]:
import pandas as pd
import numpy as np


In [3]:
"""
Creating Dataframes from Multiple CSVs in place of using jolpica-f1 API to pull data.
Data pulled from Kaggle: Formula 1 World Championship (1950 - 2024)

Start pulling data from 2006 onwards, this is when the “knockout” qualifying style happened. 

•	Circuit (Location)
•	Race (Season, Round, Race Name, Circuit info)
•	Result (Driver number, position, points, constructor, grid, laps, status, fastest lap)
•	Qualifying (Driver Pos, Driver, Constructor, ***Quali time***)
•	Constructor Standings (Points, wins, Constructor)
•	Driver Standings (Points, wins, Driver, Constructor)


•	NOT A TABLE/GIVEN:
o	Pull weather data from wiki/F1 Links
	Status Col (Dry, Wet), Temp Col

"""

'\nCreating Dataframes from Multiple CSVs in place of using jolpica-f1 API to pull data.\nData pulled from Kaggle: Formula 1 World Championship (1950 - 2024)\n\nStart pulling data from 2006 onwards, this is when the “knockout” qualifying style happened. \n\n•\tCircuit (Location)\n•\tRace (Season, Round, Race Name, Circuit info)\n•\tResult (Driver number, position, points, constructor, grid, laps, status, fastest lap)\n•\tQualifying (Driver Pos, Driver, Constructor, ***Quali time***)\n•\tConstructor Standings (Points, wins, Constructor)\n•\tDriver Standings (Points, wins, Driver, Constructor)\n\n\n•\tNOT A TABLE/GIVEN:\no\tPull weather data from wiki/F1 Links\n\uf0a7\tStatus Col (Dry, Wet), Temp Col\n\n'

In [4]:
dfcircuts = pd.read_csv("circuits.csv")
dfcircuts.sort_values('circuitId')

dfcircuts.circuitRef.unique()




array(['albert_park', 'sepang', 'bahrain', 'catalunya', 'istanbul',
       'monaco', 'villeneuve', 'magny_cours', 'silverstone',
       'hockenheimring', 'hungaroring', 'valencia', 'spa', 'monza',
       'marina_bay', 'fuji', 'shanghai', 'interlagos', 'indianapolis',
       'nurburgring', 'imola', 'suzuka', 'vegas', 'yas_marina', 'galvez',
       'jerez', 'estoril', 'okayama', 'adelaide', 'kyalami', 'donington',
       'rodriguez', 'phoenix', 'ricard', 'yeongam', 'jacarepagua',
       'detroit', 'brands_hatch', 'zandvoort', 'zolder', 'dijon',
       'dallas', 'long_beach', 'las_vegas', 'jarama', 'watkins_glen',
       'anderstorp', 'mosport', 'montjuic', 'nivelles', 'charade',
       'tremblant', 'essarts', 'lemans', 'reims', 'george', 'zeltweg',
       'aintree', 'boavista', 'riverside', 'avus', 'monsanto', 'sebring',
       'ain-diab', 'pescara', 'bremgarten', 'pedralbes', 'buddh',
       'americas', 'red_bull_ring', 'sochi', 'baku', 'portimao',
       'mugello', 'jeddah', 'losail', 

In [5]:
dfraces = pd.read_csv("races.csv")
dfraces.drop(dfraces.columns[5:], axis=1, inplace=True) #dropping time columns
dfraces = dfraces[dfraces['year'] >= 2006] #filtering races from 2006 onwards
#dfraces.head()


"""
Races from 2006 onwards are RaceID 1-70, 337-1144
"""

'\nRaces from 2006 onwards are RaceID 1-70, 337-1144\n'

In [6]:
dfresults = pd.read_csv("results.csv")
dfresults.drop("positionText", axis=1, inplace=True)
dfresults.drop("number", axis=1, inplace=True)
dfresults.drop("position", axis=1, inplace=True)
dfresults.head()


,resultId,raceId,driverId,constructorId,grid,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,5,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,11,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,3,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [7]:
dfquali = pd.read_csv("qualifying.csv")
dfquali.sort_values('q2', ascending=False)
dfquali.head()

#convert all time to seconds
def qseconds(time):
    try:
        total = time.split(':')
        mins = int(total[0]) * 60
        seconds = total[1].split('.')
        seconds_float = float(seconds[0])
        milliseconds = float('0.' + seconds[1]) if len(seconds) > 1 else 0.0
        final = mins + seconds_float + milliseconds    
        return final
    except:
        return np.nan

for q in ['q1', 'q2', 'q3']:
    dfquali[q] = dfquali[q].apply(qseconds)

#creating mean times of all quali
def meantime(q1, q2, q3):
    if pd.isna(q2):
        return (q1)
    elif pd.isna(q3):
        return (q1 + q2)/2
    else:
        return (q1 + q2 + q3)/3
    
dfquali['mean_time'] = dfquali.apply(lambda x: meantime(x[6], x[7], x[8]), axis=1)

# Create a new column for the difference of pole position?

#need to fill times for NA Q1 apperances
dflaptimes = pd.read_csv("lap_times.csv")

#filling by final laptime of the race
final_lap_times = dflaptimes.groupby(['raceId', 'driverId'])['time'].last().reset_index()
final_lap_times.rename(columns={'time': 'final_lap_time'}, inplace=True)


dfquali = pd.merge(dfquali, final_lap_times, on=['raceId', 'driverId'], how='left')

dfquali['q1'] = dfquali['q1'].fillna(dfquali['final_lap_time'])
dfquali.drop(columns='final_lap_time', inplace=True)

#drop regular times as drivers will get knocked out; this will still be taken in to account by position
dfquali.drop(dfquali.columns[6:9], axis=1, inplace=True)



/var/folders/gb/q_14btsd7vq8jrly_35pxk_m0000gn/T/ipykernel_34469/490600306.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfquali['mean_time'] = dfquali.apply(lambda x: meantime(x[6], x[7], x[8]), axis=1)


In [8]:
dfteams = pd.read_csv('constructors.csv')

dfteams.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [9]:
dfdrivers = pd.read_csv("drivers.csv")
dfdrivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [ ]:
""""
Merging DFs into One"
"""
#quali and results
result_quali = pd.merge(dfresults, dfquali, on=['raceId', 'driverId','constructorId'], how='outer')
#adding race info
race_result_quali = pd.merge(dfraces, result_quali, on=['raceId'], how='outer')
#adding circut data
cir_race_result_quali = pd.merge(dfcircuts, race_result_quali, on =['circuitId'], how='outer')
#adding driver data
driver_cir_race_result_quali = pd.merge(dfdrivers, cir_race_result_quali, on =['driverId'], how='outer')




In [36]:
#Cleaning

#removing all years not in the turbo hybrid era
df_final = driver_cir_race_result_quali[(driver_cir_race_result_quali['year'] >= 2014)]

#na quali is for those who crashed out or were not able to complete the session. since 0 would be rewarding the driver, we will give them the max time of any circut.

df_final.loc[df_final['mean_time'].isna(), 'mean_time'] = df_final['mean_time'].max()

#missing 10 races where marussia switched team names over to Manor Marussia
df_final.dropna(subset=['resultId'],axis =0 ,inplace=True)
df_final.dropna(subset=['qualifyId'],axis =0 ,inplace=True)

df_final.isna().sum()

/var/folders/gb/q_14btsd7vq8jrly_35pxk_m0000gn/T/ipykernel_34469/2374305477.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.dropna(subset=['resultId'],axis =0 ,inplace=True)
/var/folders/gb/q_14btsd7vq8jrly_35pxk_m0000gn/T/ipykernel_34469/2374305477.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.dropna(subset=['qualifyId'],axis =0 ,inplace=True)


driverId           0
driverRef          0
number_x           0
code               0
forename           0
surname            0
dob                0
nationality        0
url_x              0
circuitId          0
circuitRef         0
name_x             0
location           0
country            0
lat                0
lng                0
alt                0
url_y              0
raceId             0
year               0
round              0
name_y             0
resultId           0
constructorId      0
grid               0
positionOrder      0
points             0
laps               0
time               0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
statusId           0
qualifyId          0
number_y           0
position           0
mean_time          0
dtype: int64

In [43]:
#dropping irrelvant columns or columns that can cannot be predicted on/leak results (ie fastest lap time/rank)
data = df_final.drop(['country', 'location', 'lat', 'lng', 'alt', 'name_x', 'resultId', 'time', 'milliseconds', 'qualifyId', 'fastestLapTime', 'fastestLapSpeed', 'points', 'fastestLap', 'number_x', 'number_y', 'forename', 'surname', 'url_x', 'url_y', 'name_y', 'position', 'rank', 'circuitId', 'raceId',  ], axis =1)

#rank is fastest lap rank
#pos order is the final pos
#grid is starting grid

#remaining items: Clean drivers, add constructor labels, add status labels

#cleaning driver cols (driverId, driverRef, code) and adding Age as a feature via DOB

data = data.drop(['driverId', 'driverRef'], axis=1) #code, nationality is left to encode with OHE

data['dob'] = pd.to_datetime(data.dob)
data['driver_age'] = data.year - data.dob.dt.year
data.drop(['dob'], axis=1, inplace= True)

#adding constructor names to encode and dropping those that dont fit in the turbo hybrid era
temp = data.copy()
data = pd.merge(dfteams, temp, on=['constructorId'], how='outer').drop(['nationality_x','url', 'constructorRef'], axis =1)
data = data.dropna(subset=['code'])
data.drop(['constructorId'], axis=1, inplace=True)

#combining constructor names as teams have remained the same but have changed title sponsors
def nameupdate(team):
    if team == "Renault" or team == 'Lotus F1':
        return 'Alpine F1 Team'
    elif team == "Toro Rosso" or team == 'AlphaTauri':
        return 'RB F1 Team'
    elif team == "Force India"or team == 'Racing Point':
        return 'Aston Martin'
    elif team == "Alfa Romeo":
        return 'Sauber'
    elif team == "Marussia":
        return 'Manor Marussia'
    else:
        return team
    
data['name'] = data['name'].apply(nameupdate)

#creating features for status to show if driver or car dnf
no_fault = [11,88,45,55,53,111,112,116,50,114,124,12,127,120,115,119,117,113,58,118,13,123,134,14,128,122,125,133,15,16,17,18,19,7,1]
driver_dnf = [3,41,4,130,137,97,81,104]
car_dnf = [91,109,84,135,23,83,8,141,87,102,138,24,2,99,28,100,30,79,10,40,5,56,129,132,96,43,107,66,33,74,95,106,32,48,71,69,6,86,64,42,9,80,139,98,73,82,72,121,26,62,90,44,70,108,51,94,60,25,68,63,75,131,29,21,65,35,31,78,93,89,136,105,20,85,38,126,22,39,37,49,7,101,27,59,140,92,76,47,110,34,103,36,67,61,46,54]

data['driver_dnf'] = data['statusId'].isin(driver_dnf).astype(int)
data['car_dnf'] = data['statusId'].isin(car_dnf).astype(int)
data.drop(['statusId'], axis=1, inplace=True)
#data.dtypes

data.name.unique()

array(['McLaren', 'Williams', 'Alpine F1 Team', 'RB F1 Team', 'Ferrari',
       'Red Bull', 'Aston Martin', 'Sauber', 'Mercedes', 'Manor Marussia',
       'Caterham', 'Haas F1 Team'], dtype=object)

In [47]:
#final renaming of cols and csv export

data = data.rename(columns={
    'name' : 'team_name',
    'nationality_y' : 'driver_nat',
    'grid' : 'starting_pos',
    'positionOrder' : 'finishing_pos',
    'mean_time' : 'quali_mean'
})


data.to_csv('f1_cleaned.csv', index=False)